In [ ]:
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVC
import operator
import googlefinance.client as gfc
from matplotlib import pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

import quandl

%matplotlib notebook

The goal of this notebook is to create time series regression models using some regression models as well as "Forest" models (both regression and deep learning versions). 

The features for this part of the research are going to be extracted using data freely available in Python via Quandl and pandas Yahoo finance support. 

- NASDAQ Composite
- Dow Jones Industrial Average
- Frankfurt DAX
- London FTSE-100
- Paris CAC 40
- Tokyo Nikkei-225
- Hong Kong Hang Seng
- Australia ASX-200

The features will be extracted from the major market indicators above will have the following format:

- Date : in days

- Open : price of the stock at the opening of the trading (in US dollars)
- High : highest price of the stock during the day (in US dollars)
- Low : lowest price of the stock during the trading day (in US dollars)
- Close : price of the stock at the closing of the trading (in US dollars)
- Volume : amount of stocks traded (in US dollars)
- Adj Close : price of the stock at the closing of the trading adjusted with dividends (in US dollars)

In [ ]:
# Get googlefinance major global ETF data

period = "12M"

params = [
    {'q': "IXIC",
     'x': 'INDEXNASDAQ'
    },
    {'q': '.DJI',
      'x': 'INDEXDJX'  
    },
    {'q':'DAX',
     'x':'INDEXDB'   
    },
    {'q': 'UKX',
      'x':'INDEXFTSE'   
    },
    {'q': 'PX1',
      'x': 'INDEXEURO'   
    },
    {'q': 'NI225',
      'x': 'INDEXNIKKEI'  
    },
    {'q': 'HSI',
      'x': 'INDEXHANGSENG'  
    },
    {'q': 'XJO',
      'x':'INDEXASX'   
    },
    {'q': "SPY",
     'x': 'NYSEARCA'
    }
]
feats1 = gfc.get_prices_data(params, period)
# Clear the null volume data
j = feats1.pop('IXIC_Volume')
j = feats1.pop('XJO_Volume')
feats1.shape

In [ ]:
# Get Interest rate yield curve pct_change
# Get inflation pct_change
# Get yields of ETFs (2nd stage)
# Get US GDP pct_change
# Get correlations of the signals listed with respct to SPY (or alternative ETFs listed in project)



# Currently, some placeholder data from quandl, going to replace with data mentioned above
quandl.ApiConfig.api_key = "a1LZ4iFxZS6mzZUsf28u"
feats2 = quandl.get('BEA/NIPA_2_8_7_M', start_date='2016-08-31',
                   end_date='2017-08-31')
feats1 = feats1.dropna()

In [ ]:
x = feats1
y = feats2

In [ ]:
y = y.resample('1D', 'ffill').reindex(x.index)

In [ ]:
for i in y.columns:
    x[i] = y[i]

In [ ]:
x.head()
feats1 = x

In [ ]:
feats1 = feats1.dropna()
y = feats1.pop('SPY_Close')
X = feats1

In [ ]:
X.tail()

In [ ]:
y = y.shift(1).dropna()

In [ ]:
X = X[:-1]

In [ ]:
X.shape, y.shape

In [ ]:
# Use 95% to train of the 12 month data set

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
training_range = int(X.shape[0] *.95)
X_train = X[0:training_range]
y_train = y[0:training_range]

X_test = X[training_range: ]
y_test = y[training_range: ]

In [ ]:
X_train.head()

In [ ]:
# The next day prediction
y_train.head()

# Ordinary Least Square

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
y_predicted = reg.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_predicted, label='Predicted SPY')
ax.set_title('OLS Prediction for SPY using 1 year period')
plt.legend()



reg.score(X_test, y_test)

# RidgeCV Method

In [ ]:
ridge = linear_model.RidgeCV()
ridge.fit(X_train, y_train)
y_ridge_predicted = ridge.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_ridge_predicted, label='Predicted SPY')
ax.set_title('RidgeCV Prediction for SPY using 1 year period')
plt.legend()

ridge.score(X_test, y_test)

# Lasso 

In [ ]:
lasso = linear_model.Lasso(alpha = 1e-5)
lasso.fit(X_train, y_train)

y_predicted = lasso.predict(X_test)

fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(y_predicted, label='Predicted SPY')
ax.set_title('Lasso Prediction for SPY using 1 year period')
plt.legend()

print(y_predicted[4], y_test.values[4])

lasso.score(X_test, y_test)

# Random Forest Regressor

In [ ]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)
regr_predicted = regr.predict(X_test)
fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual SPY')
ax.plot(regr_predicted, label='Predicted SPY')
ax.set_title('Random Forest Regression for SPY using 1 year period')
plt.legend()

regr.score(X_test, y_test)
